# Exercícios do curso de Pandas avançado

## Começando o trabalho

### Carregando arquivos JSON

Em uma base de dados que relaciona o nome da pessoa e seus dados referentes à idade, peso e altura, considere a variável data_json abaixo:

In [4]:
import pandas as pd
data_json = '{"Rita": {"Idade": 24, "Peso": 62, "Altura": 1.65}, "Zeca": {"Idade": 32, "Peso": 80, "Altura": 1.82}}'

Indique a opção que mostra os códigos que produzem os seguintes resultados:
    
**Tabela A**

| | Idade | Peso | Altura |
|---|---|---|---|
|Rita | 24 | 62 | 1.65 |
|Zeca | 32 | 80 | 1.82 |

**Tabela B**

|	|Rita|	Zeca|
|---|---|---|
|Idade|	24|	32|
|Peso	|62|	80|
|Altura	|1.65|	1.82|

In [5]:
dados = pd.read_json(
    path_or_buf = data_json,
    orient = 'index'
)
dados

,Idade,Peso,Altura
Rita,24,62,1.65
Zeca,32,80,1.82


In [7]:
dados = pd.read_json(
    path_or_buf = data_json
)
dados

,Rita,Zeca
Idade,24.00,32.00
Peso,62.00,80.00
Altura,1.65,1.82


### Carregando arquivos Excel

No arquivo Excel (XLSX) disponibilizado (bairros.xlsx) temos duas planilhas. Na planilha “Residencial X Comercial”, temos os valores do m2 para imóveis comerciais e residenciais nos bairros da cidade do Rio de Janeiro. Temos também a divisão regional das informações por zonas da cidade.

Utilizando esta planilha e o método `read_excel()` do pandas obtenha o seguinte DataFrame:

|Zonas	|Bairros|	Residencial|	Comercial|
|---|---|---|---|
|Sul	|Botafogo	|14002	|7972|
||Catete	|15232|	6259|
||Copacabana	|23318|	9355|
|... |...|...|...|

Observe que foram selecionadas apenas as linhas dos bairros pertencentes à zona sul da cidade.

Marque a opção que apresenta o código necessário para criar o DataFrame acima.

In [15]:
pd.read_excel(
    io = 'dados/bairros.xlsx',
    sheet_name = 'Residencial X Comercial',
    #header = 1,
    names = ['Zonas', 'Bairros', 'Residencial', 'Comercial'],
    index_col = [0, 1],
    usecols = 'B:E',
    skiprows = 18,
    nrows = 17
)

Residencial  Comercial
Zonas Bairros                                
Sul   Botafogo               14002       7972
      Catete                 15232       6259
      Copacabana             23318       9355
      Cosme Velho            10320       8177
      Flamengo               19636       7135
      Gávea                  13506       8211
      Humaitá                10603       5039
      Ipanema                15965       7293
      Jardim Botânico        17243       8095
      Lagoa                  24982       6584
      Laranjeiras            17307       7745
      Leblon                 13815       8125
      Leme                   15815       8587
      Rocinha                15110       9023
      São Conrado            20727       9587
      Urca                   22876       6607
      Vidigal                23470       5424

## Transformando e tratando os dados

### Normalizando nossos dados

Estamos trabalhando com uma base de dados que relaciona o nome dos alunos, suas idades e medidas (peso e altura). Considere a variável data_json abaixo:

In [23]:
data_json = '{"alunos": [{"Nome": "Rita", "Info": {"Idade": 24, "Medidas": {"Peso": 62, "Altura": 1.65}}}, {"Nome": "Zeca", "Info": {"Idade": 32, "Medidas": {"Peso": 80, "Altura": 1.82}}}]}'

Utilizando os métodos aprendidos, assinale a alternativa que produz como resultado o seguinte DataFrame:


||Nome	|Info_Idade|	Info_Medidas|
|---|---|---|---|
|0	|Rita|	24|	{'Peso': 62, 'Altura': 1.65}|
|1|	Zeca|	32|	{'Peso': 80, 'Altura': 1.82}|


In [30]:
df_json = pd.read_json(data_json)
df_json

,alunos
0,"{'Nome': 'Rita', 'Info': {'Idade': 24, 'Medida..."
1,"{'Nome': 'Zeca', 'Info': {'Idade': 32, 'Medida..."


In [33]:
pd.json_normalize(data = df_json.alunos, sep = '_', max_level = 1)

,Nome,Info_Idade,Info_Medidas
0,Rita,24,"{'Peso': 62, 'Altura': 1.65}"
1,Zeca,32,"{'Peso': 80, 'Altura': 1.82}"


### Praticando o uso de métodos de strings

Em algumas bases de dados encontramos colunas de informações no formato de texto (strings) que podem apresentar, em seu conteúdo, mais de um tipo de informação. Nestes casos torna-se necessário utilizar métodos específicos para tentar separar estas informações em colunas distintas. Vamos treinar a utilização destes métodos com a variável data_string abaixo:

In [1]:
data_string = "#-> Churrasqueira | Sauna | Mobiliado | Piscina <-#"

Utilizando os métodos de string que aprendemos no último vídeo, assinale os itens que retornam o seguinte resultado:

`['Churrasqueira', 'Sauna', 'Mobiliado', 'Piscina']`

In [3]:
data_string[4:-4].split(' | ')

['Churrasqueira', 'Sauna', 'Mobiliado', 'Piscina']

In [6]:
data_string.strip('#->< ').split(' | ')

['Churrasqueira', 'Sauna', 'Mobiliado', 'Piscina']

### Utilizando o método filter

Em algumas situações, durante a fase de exploração dos dados, o profissional de data science precisa executar alguns filtros no dataset. Este procedimento pode ter como objetivo a aplicação de certos tipos de tratamento em um grupo específico de colunas ou a geração de tabulações específicas entre certas variáveis.

O método `filter` que vimos possibilita a criação de subconjunto das linhas ou colunas de um DataFrame de acordo com os rótulos dos eixos especificados. Observe que este método não filtra um DataFrame em seu conteúdo. O filtro é apenas aplicado aos rótulos dos índices ou colunas.

Considere o DataFrame `df`:

In [9]:
import pandas as pd
dados = {
    "alunos": ["Rita", "Lucas", "Zeca", "Ana"], 
    "idade": [10, 12, 11, 10], 
    "medidas_altura": [1.3, 1.5, 1.45, 1.28], 
    "medidas_peso": [42, 50, 45, 38]
}

df = pd.DataFrame(dados)

Utilizando o método `filter`, assinale a opção que retorna um DataFrame com apenas as colunas "medidas_altura" e "medidas_peso".

In [12]:
df.filter(like = 'medidas')

,medidas_altura,medidas_peso
0,1.30,42
1,1.50,50
2,1.45,45
3,1.28,38


## Combinando conjuntos de dados

### Os métodos append e concat

Em certos momentos, durante a fase de exploração dos dados, precisamos juntar em nosso dataset novas informações. Isso é bastante comum em projetos de data science que precisam ser atualizados constantemente devido a ocorrência de novos registros sobre o objeto de estudo. Para este tipo de trabalho o pandas disponibiliza alguns métodos como o `append` e o `concat` que conhecemos.

Assinale a alternativa que apresenta a forma correta de se utilizar os métodos `append` e `concat` do `pandas` para obter o seguinte resultado:


||A|	B|
|---|---|---|
|0|	1|	1|
|1|	2|	4|
|2|	3|	9|
|3|	4|	16|
|4|	5|	25|
|5|	6|	36|

Para isso utilize os dois DataFrames resultantes do código abaixo:

In [3]:
import pandas as pd
df_A = pd.DataFrame({'A': [1, 2, 3], "B": [1, 4, 9]})
df_B = pd.DataFrame({'A': [4, 5, 6], "B": [16, 25, 36]})

In [4]:
pd.concat([df_A, df_B], ignore_index = True)

,A,B
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25
5,6,36


In [5]:
df_A.append(df_B, ignore_index = True)

C:\Users\beatr\AppData\Local\Temp\ipykernel_16144\1994273176.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_A.append(df_B, ignore_index = True)


,A,B
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25
5,6,36


### O parâmetro sort dos métodos append e concat

Os métodos `append` e `concat` possuem o parâmetro `sort` que é um booleano que vem, por *padrão*, configurado como `False`. Quando configurado como `True` classifica os eixos (colunas no caso do `append` e colunas ou linhas no caso do `concat`) caso ainda não estejam alinhados.

Para entender melhor como essa classificação funciona vamos a um exemplo prático. Considere os DataFrames **df_A** e **df_B**:

In [6]:
df_A = pd.DataFrame({'A': [1, 2, 3], 'B': [1, 4, 9]})

In [7]:
df_B = pd.DataFrame({'B': [16, 25, 36], 'A': [4, 5, 6]})

Executando a linha de código abaixo, qual seria o DataFrame resultante?

In [8]:
df_B.append(df_A, ignore_index=True, sort=True)

C:\Users\beatr\AppData\Local\Temp\ipykernel_16144\3686226822.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_B.append(df_A, ignore_index=True, sort=True)


,A,B
0,4,16
1,5,25
2,6,36
3,1,1
4,2,4
5,3,9


### Substituindo valores

A Series nomes contém os nomes de alguns assuntos interessantes quando falamos sobre análise de dados:

In [10]:
nomes = pd.Series(['Data Science', 'Big Data', 'DS', 'Machine Learning', 'ML'])
nomes

0        Data Science
1            Big Data
2                  DS
3    Machine Learning
4                  ML
dtype: object

Assinale os códigos que, com os métodos vistos, geram a Series abaixo:

||0|
|---|---|
|0|	Data Science|
|1|	Big Data|
|2|	Machine Learning|

Observe que 'DS' e 'ML' são siglas para 'Data Science' e 'Machine Learning', respectivamente.

In [14]:
pd.Series(nomes.replace({'DS': 'Data Science', 'ML': 'Machine Learning'}).unique())

0        Data Science
1            Big Data
2    Machine Learning
dtype: object

In [16]:
# outras possibilidades (colocadas como alternativas do exercício)
pd.Series(nomes.replace(['DS', 'ML'], ['Data Science', 'Machine Learning']).unique())

0        Data Science
1            Big Data
2    Machine Learning
dtype: object

In [17]:
pd.Series(nomes.replace('DS', 'Data Science').replace('ML', 'Machine Learning').unique())

0        Data Science
1            Big Data
2    Machine Learning
dtype: object